In [43]:
import numpy as np
import pandas as pd
import tensorflow

from keras.models import Sequential
from keras.layers import Dense ,SimpleRNN, LSTM ,GRU ,TimeDistributed #(다대다 구성에 필요)   

In [44]:
import FinanceDataReader as fdr

df = fdr.DataReader('005930','2000-01-01')
df = df[['Close','High','Volume']].reset_index()
df

,Date,Close,High,Volume
0,2000-01-04,6110,6110,1483967
1,2000-01-05,5580,6060,1493604
2,2000-01-06,5620,5780,1087810
3,2000-01-07,5540,5670,806195
4,2000-01-10,5770,5770,937615
...,...,...,...,...
5663,2022-12-09,60400,60600,10529632
5664,2022-12-12,59500,60000,9036472
5665,2022-12-13,59700,60300,9040873
5666,2022-12-14,60500,60600,8207485


In [45]:
df.drop(columns=["Date"], inplace=True)
df

,Close,High,Volume
0,6110,6110,1483967
1,5580,6060,1493604
2,5620,5780,1087810
3,5540,5670,806195
4,5770,5770,937615
...,...,...,...
5663,60400,60600,10529632
5664,59500,60000,9036472
5665,59700,60300,9040873
5666,60500,60600,8207485


In [46]:
##batch size, seq len, hidden size(=특성 데이터 개수) = (-1,10,3)  
##으로  
##종가 10일을 예측  

In [47]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [48]:
x,y = make_dataset( df[["Close","High","Volume"]], df[["Close"]], 10)
#label이 a인 이유는 결국에 a를 이용해서 a 자기 자신을 예측하고 싶기 때문
#만약 y를 주게되면 8을 가져와버려서 shift해서 가져오면 됨
#window size = shift = 3이라면 y를 그대로 쓸 수 있음

#즉, 여기서 y는 윈도우 사이즈 3일때니까 다른 사이즈로 하고 싶으면 커스터마이징 해서 y를 만들어줘야해서
#a를 사용한 것

In [49]:
x= x.reshape(-1,10,3)
x.shape

(5658, 10, 3)

In [50]:
y.shape

##batch size, seq len, hidden size(=특성 데이터 개수)

(5658, 1)

In [51]:
model = Sequential()
model.add(LSTM(50,activation='relu',input_shape=(10,3), return_sequences=True))
model.add(LSTM(30,activation='relu',input_shape=(10,3)))
model.add(Dense(units=3))
model.compile(loss='mse',optimizer='adam')
model.fit(x,y,epochs=1000)

Epoch 1/1000
177/177 [==============================] - 3s 7ms/step - loss: 35826872320.0000
Epoch 2/1000
177/177 [==============================] - 1s 6ms/step - loss: 13965556736.0000
Epoch 3/1000
177/177 [==============================] - 1s 6ms/step - loss: 1492585728.0000
Epoch 4/1000
177/177 [==============================] - 1s 6ms/step - loss: 3632322816.0000
Epoch 5/1000
177/177 [==============================] - 1s 6ms/step - loss: 3235435264.0000
Epoch 6/1000
177/177 [==============================] - 1s 6ms/step - loss: 1035472064.0000
Epoch 7/1000
177/177 [==============================] - 1s 6ms/step - loss: 989464320.0000
Epoch 8/1000
177/177 [==============================] - 1s 6ms/step - loss: 692842816.0000
Epoch 9/1000
177/177 [==============================] - 1s 6ms/step - loss: 593282816.0000
Epoch 10/1000
177/177 [==============================] - 1s 6ms/step - loss: 449337984.0000
Epoch 11/1000
177/177 [==============================] - 1s 6ms/step - loss: 8982

In [52]:
model.predict(x)

177/177 [==============================] - 1s 2ms/step


array([[ 5933.996 ,  5933.947 ,  5934.0537],
       [ 5966.652 ,  5966.6016,  5966.711 ],
       [ 5916.973 ,  5916.924 ,  5917.0303],
       ...,
       [58584.53  , 58583.086 , 58586.242 ],
       [59736.402 , 59734.926 , 59738.145 ],
       [60014.96  , 60013.477 , 60016.715 ]], dtype=float32)

In [56]:
pred_10 = model.predict(x)[-10:,:]
pred_10_close=[]
for i in range(len(pred_10)):
    pred_10_close.append(pred_10[i][0])
pred_10_close

177/177 [==============================] - 0s 2ms/step


[60887.367,
 60059.27,
 59732.3,
 58153.633,
 59543.453,
 58131.03,
 59808.938,
 58584.53,
 59736.402,
 60014.96]